# Final Project for INFO 6350
## Net ID: yz2685

In [1]:
# import our libraries
import requests
import urllib
from bs4 import BeautifulSoup
import simplejson as json
from urllib.request import Request, urlopen
from fake_useragent import UserAgent

In [2]:
# define the base url needed to create the file url.
base_url = r"https://www.sec.gov"

header = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
}
# content = requests.get(documents_url, headers=header)
# print(content)

# Grabbing the Financial Statements

In [3]:
# create the list to hold the statement urls
statements_url = []

for report_dict in master_reports:
    
    # define the statements we want to look for.
    item1 = r"Consolidated Balance Sheets"
    item2 = r"Consolidated Statements of Operations and Comprehensive Income (Loss)"
    item3 = r"Consolidated Statements of Cash Flows"
    item4 = r"Consolidated Statements of Stockholder's (Deficit) Equity"
    
    # store them in a list.
    report_list = [item1, item2, item3, item4]
    
    # if the short name can be found in the report list.
    if report_dict['name_short'] in report_list:
        
        # print some info and store it in the statements url.
        print('-'*100)
        print(report_dict['name_short'])
        print(report_dict['url'])
        
        statements_url.append(report_dict['url'])

# Scraping the SEC Query Page

In [4]:
# base URL for the SEC EDGAR browser
endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

# define our parameters dictionary
param_dict = {'action':'getcompany',
              # 'CIK':'1318605', # Tesla
              # 'CIK':'320193', # Apple
              'CIK':'1018724', # Amazon
              'type':'10-k',
              'dateb':'20230101',
              'owner':'exclude',
              'start':'',
              'output':'',
              'count':'100'}

# request the url, and then parse the response.
response = requests.get(url = endpoint, params = param_dict, headers=header)
# response = requests.get(url = endpoint, params = param_dict)
soup = BeautifulSoup(response.content, 'html.parser')

# Let the user know it was successful.
print('Request Successful')
print(response.url)

Request Successful
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1018724&type=10-k&dateb=20230101&owner=exclude&start=&output=&count=100


In [5]:
doc_table = soup.find_all(class_ = "blueRow")
# print(doc_table)

In [6]:
data = soup.find_all(class_='blueRow')

list_10k = []

for i, row in enumerate(data): 
    for a in data[i].find_all('a', href=True):
        url = a['href']
        if (url.startswith('/Archives/edgar/')):
            list_10k.append("https://www.sec.gov"+ url)
            print("https://www.sec.gov"+ url)

https://www.sec.gov/Archives/edgar/data/1018724/000101872421000004/0001018724-21-000004-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/0001018724-19-000004-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/0001018724-15-000006-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312513028520/0001193125-13-028520-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312511050136/0001193125-11-050136-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312510016098/0001193125-10-016098-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312508024707/0001193125-08-024707-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312506034166/0001193125-06-034166-index.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312504029488/0001193125-04-029488-index.htm
https://www.sec.gov/

# Scraping Company Page

In [7]:
# get the links to the 10k reports 

def get10kPages(url):
    response = requests.get(url = url, headers=header)
    soup10k = BeautifulSoup(response.content, 'html.parser')
    # print(response)
    # print(response.url)
    return response.url

In [8]:
list_of_links = []
for link in list_10k:
    list_of_links.append(get10kPages(link))
print(list_of_links)

['https://www.sec.gov/Archives/edgar/data/1018724/000101872421000004/0001018724-21-000004-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/0001018724-19-000004-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/0001018724-17-000011-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/0001018724-15-000006-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312513028520/0001193125-13-028520-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312511050136/0001193125-11-050136-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312510016098/0001193125-10-016098-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312508024707/0001193125-08-024707-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312506034166/0001193125-06-034166-index.htm', 'https://www.sec.gov/Archives/edgar/data/1018724/000119312504029488/0001193125-04-029488-i

In [9]:
# get the url for 10k report 
list_of_10ks = []

def get10kLinks(url):
    response = requests.get(url = url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    # for a in soup.find_all('a', href=True):
        # url = a['href']
        # print(url)
        
    suffix = "htm";

    tables = soup.find('table')
    rows = tables.find_all('tr')
    if len(rows) > 0:
        row10k = rows[1] # row 1 has link to 10k report
        # print(row10k)
        for a in row10k.find_all('a', href=True):
            url = a['href']
            if url.endswith(suffix):
                list_of_10ks.append("https://www.sec.gov"+ url)
                print("https://www.sec.gov"+ url)

In [10]:
for url in list_of_links:
    get10kLinks(url)
    
# list_of_10ks

https://www.sec.gov/ix?doc=/Archives/edgar/data/1018724/000101872421000004/amzn-20201231.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/amzn-20181231x10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/amzn-20161231x10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/amzn-20141231x10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312513028520/d445434d10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312511050136/d10ka.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312510016098/d10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312508024707/d10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312506034166/d10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000119312504029488/d10k.htm
https://www.sec.gov/Archives/edgar/data/1018724/000103221002000059/d10k405.htm


In [79]:
from selenium import webdriver

driver = webdriver.Firefox()

In [12]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [82]:
# create a function that will scrap the 10k report 
# import chromedriver_binary  # Adds chromedriver binary to path
import time
import re

data = []
y_labal = []

def scrap10k(url):
    driver.get(url)

    time.sleep(3) # give browser some time to load the js 

    html = driver.page_source
    sp = BeautifulSoup(html)
    
    text = ""
    for d in sp.find_all(text=True):
        text += d.get_text()
    
    # cleaning 
    # print(text)
    text = text.replace(u'\xa0', u' ').lower()

    # extract y_label - net profit margin ratio = (total net sales - cost of sales) / total net sales
    tds = []
    for td in sp.find_all("td"):
        tds.append(td.text)
                
            
    eps = ""
    for i, word in enumerate(tds):
        if "earnings per share" in word.lower():
            # print(i) # i: the index of first eps
            for j, word in enumerate(tds[i:]):
                word = word.replace(u'\xa0', u' ').lower()
                sign = 1
                if word.find("(") != -1:
                    sign = -1
                eps = re.findall('\d+\.*\d+', word) * sign
                if len(eps) > 0:
                    print(eps[0])
                    break
            break
    
    
    # extract risk factor sections only
    print(url)
    start = find_nth(text, "item 1a.", 2)
    print("start index", start)
    end = find_nth(text, "item 1b.", 2)
    print("end index", end)
    substring = text[start:end]
    if len(substring) > 0:
        data.append(substring)
        data.append(eps[0])
    

In [70]:
# test 
# scrap10k(list_of_10ks[0])

In [83]:
%%time
for report in list_of_10ks:
    scrap10k(report)

5.01
https://www.sec.gov/ix?doc=/Archives/edgar/data/1018724/000101872421000004/amzn-20201231.htm
start index 50652
end index 100536
1.28
https://www.sec.gov/Archives/edgar/data/1018724/000101872419000004/amzn-20181231x10k.htm
start index 17482
end index 61930
0.60
https://www.sec.gov/Archives/edgar/data/1018724/000101872417000011/amzn-20161231x10k.htm
start index 17544
end index 61696
0.60
https://www.sec.gov/Archives/edgar/data/1018724/000101872415000006/amzn-20141231x10k.htm
start index 16615
end index 59674
1.39
https://www.sec.gov/Archives/edgar/data/1018724/000119312513028520/d445434d10k.htm
start index 16301
end index 56333
https://www.sec.gov/Archives/edgar/data/1018724/000119312511050136/d10ka.htm
start index -1
end index -1
2.08
https://www.sec.gov/Archives/edgar/data/1018724/000119312510016098/d10k.htm
start index 17060
end index 55919
1.15
https://www.sec.gov/Archives/edgar/data/1018724/000119312508024707/d10k.htm
start index 25571
end index 65356
0.77
https://www.sec.gov/A

In [ ]:
len(data)

16

In [24]:
data[7]

# Analysis

In [17]:
# import unicodedata

# text = unicodedata.normalize('NFKD', text.replace("\'", "'").replace("\ in\ form", " inform").replace("\n", " ").lower().strip())

In [18]:
# print(text)
# dev_docs = [doc for doc in text.split('\n')] 

In [19]:
# from   sklearn.feature_extraction.text import TfidfVectorizer
# from   nltk.corpus import stopwords
# import string

# # Making stopwords list
# stoplist = stopwords.words('english')
# for el in [i for i in string.punctuation]:
#     stoplist.append(el)

# # Set up vectorizer
# vectorizer = TfidfVectorizer(
#     encoding='utf-8',
#     min_df=1, # include words that occur in as few as a single document, 1 = a single document
#     max_df=1.0, # include words that occur in as many as all documents, 1.0 = 100% = all documents
#      binary=False,
#     norm='l2',
#     stop_words = stoplist,
#     use_idf=True
# )

# X = vectorizer.fit_transform(dev_docs)

In [20]:
# print(X.shape)

In [21]:
# print(X)

# References

https://stackoverflow.com/questions/1883980/find-the-nth-occurrence-of-substring-in-a-string